# More detailed steps can be found in the *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow* textbook

# 1- Explore the Data
**Note**:

- try to get insights from a field expert for these steps.

### a. Create a copy of the data for exploration (sampling it down to a manageable size if necessary).

### b. Create a Jupyter notebook to keep a record of your data exploration.

### c. Study each attribute and its characteristics:
• Name

• Type (categorical, int/float, bounded/unbounded, text, structured, etc.)

• % of missing values

• Noisiness and type of noise (stochastic, outliers, rounding errors, etc.)

• Usefulness for the task

• Type of distribution (Gaussian, uniform, logarithmic, etc.)

### d. For supervised learning tasks, identify the target attribute(s).

### e. Visualize the data.

### f. Study the correlations between attributes.

### g. Study how you would solve the problem manually.

### h. Identify the promising transformations you may want to apply.

### i. Identify extra data that would be useful.

### j. Document what you have learned.


# 2- Prepare the Data
**Notes**:

• Work on copies of the data (keep the original dataset intact).

• Write functions for all data transformations you apply, for five reasons:
* So you can easily prepare the data the next time you get a fresh dataset

* So you can apply these transformations in future projects.

* To clean and prepare the test set

* To clean and prepare new data instances once your solution is live

* To make it easy to treat your preparation choices as hyperparameters

### a. Clean the data:
• Fix or remove outliers (optional).

• Fill in missing values (e.g., with zero, mean, median…) or drop their rows (or
columns).

### b. Perform feature selection (optional):
• Drop the attributes that provide no useful information for the task.

### c. Perform feature engineering, where appropriate:
• Discretize continuous features.

• Decompose features (e.g., categorical, date/time, etc.).

• Add promising transformations of features (e.g., log(x), sqrt(x), etc.).

• Aggregate features into promising new features.

### d. Perform feature scaling:
• Standardize or normalize features.

# 3- Shortlist Promising Models
**Notes**:

• If the data is huge, you may want to sample smaller training sets so you can train
many different models in a reasonable time (be aware that this penalizes complex
models such as large neural nets or random forests).

• Once again, try to automate these steps as much as possible.

### a. Train many quick-and-dirty models from different categories 
(e.g., linear, naive Bayes, SVM, random forest, neural net, etc.) using standard parameters.

### b. Measure and compare their performance:
• For each model, use N-fold cross-validation and compute the mean and stan‐
dard deviation of the performance measure on the N folds.

### c. Analyze the most significant variables for each algorithm.

### d. Analyze the types of errors the models make:
• What data would a human have used to avoid these errors?

### e. Perform a quick round of feature selection and engineering.

### f. Perform one or two more quick iterations of the five previous steps.

### g. Shortlist the top three to five most promising models, preferring models that make different types of errors.

# 4- Fine-Tune the System
**Notes**:

• You will want to use as much data as possible for this step, especially as you move
toward the end of fine-tuning.

• As always, automate what you can.

### a. Fine-tune the hyperparameters using cross-validation:

• Treat your data transformation choices as hyperparameters, especially when
you are not sure about them (e.g., if you’re not sure whether to replace missing
values with zeros or with the median value, or to just drop the rows).

• Unless there are very few hyperparameter values to explore, prefer random
search over grid search. If training is very long, you may prefer a Bayesian
optimization approach.

### b. Try ensemble methods. Combining your best models will often produce better performance than running them individually.

### c. Once you are confident about your final model, measure its performance on the test set to estimate the generalization error.